#  Deep Learning Hyperparameter Tuning (Fast)
Using `RandomizedSearchCV` and `KerasTuner` to optimize deep learning models for online fraud detection.

## Step 1: Load and Prepare Data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load dataset
df = pd.read_csv("enhanced_online_fraud_dataset.csv")
X = df.drop(columns=['isFraud', 'nameOrig', 'nameDest', 'step'])
y = df['isFraud']

# Train-test split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Step 2: Define Model for KerasTuner

In [ ]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units', min_value=32, max_value=256, step=32), activation='relu', input_shape=(X_train_scaled.shape[1],)))
    if hp.Boolean("dropout"):
        model.add(Dropout(rate=hp.Float("dropout_rate", 0.2, 0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=hp.Choice("optimizer", ["adam", "rmsprop"]),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

## Step 3: Run KerasTuner RandomSearch

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='kt_tuning',
    project_name='fraud_dl_tune'
)

tuner.search(X_train_scaled, y_train, epochs=10, validation_split=0.2, batch_size=256, verbose=1)

## Step 4: Gettin Best Model & Save It

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate
loss, acc = best_model.evaluate(X_test_scaled, y_test, verbose=0)
print("Test Accuracy:", acc)

# Save model
best_model.save("fraud_dl_model.h5")
print("✅ Model saved as 'fraud_dl_model.h5'")

## Step 5: prediction usin the uploaded data

In [ ]:
# Load model for inference
from tensorflow.keras.models import load_model

model = load_model("fraud_dl_model.h5")

# Pick some fraud examples
fraud_samples = df[df['isFraud'] == 1].sample(3, random_state=42)
fraud_features = fraud_samples.drop(columns=['isFraud', 'nameOrig', 'nameDest', 'step'])

# Scale and predict
scaled_fraud = scaler.transform(fraud_features)
predictions = model.predict(scaled_fraud)

# Output predictions
for i, prob in enumerate(predictions):
    print(f"Sample {i+1} - Fraud Probability: {prob[0]:.4f}")